In [571]:
# Import python libraries for data wrangling
import pandas as pd
import numpy as np
from urllib.parse import urlparse

In [572]:
shoes_one = pd.read_csv("Datafiniti_Womens_Shoes.csv")
shoes_two = pd.read_csv("Datafiniti_Womens_Shoes_Jun19.csv")
#shoes_three =  pd.read_csv("7210_1.csv")

In [574]:
print("There are {} rows and {} columnns in the dataset".format(shoes_one.shape[0],   shoes_one.shape[1]))
print("There are {} rows and {} columnns in the dataset".format(shoes_two.shape[0],   shoes_two.shape[1]))

There are 10000 rows and 34 columnns in the dataset
There are 10000 rows and 32 columnns in the dataset


In [576]:
print("There are {} null values in the sourceURLs column".format(shoes_one['prices.sourceURLs'].isnull().sum()))
print("There are {} null values in the merchant column".format(shoes_one['prices.merchant'].isna().sum()))
print("There are {} null values in the sourceURLs column".format(shoes_two['prices.sourceURLs'].isna().sum()))
print("There are {} null values in the merchant column".format(shoes_two['prices.merchant'].isna().sum()))

There are 0 null values in the sourceURLs column
There are 9565 null values in the merchant column
There are 0 null values in the sourceURLs column
There are 10000 null values in the merchant column


In [577]:
shoes_one.drop_duplicates(subset ="id",keep = False, inplace = True)
shoes_one = shoes.reset_index(drop = True)
shoes_two.drop_duplicates(subset ="id",keep = False, inplace = True)
shoes_one = shoes.reset_index(drop = True)

In [578]:
shoes_one.head(10)

,id,brand,categories,prices.amountMax,prices.amountMin,prices.size,prices.sourceURLs
0,AVpfEf_hLJeJML431ueH,Naturalizer,"Clothing,Shoes,Women's Shoes,All Women's Shoes...",55.99,55.99,S,https://www.overstock.com/Clothing-Shoes/Women...
1,AVpi74XfLJeJML43qZAc,MUK LUKS,"Clothing,Shoes,Women's Shoes,Women's Casual Sh...",47.00,35.25,6,https://www.walmart.com/ip/MUK-LUKS-Womens-Jan...
2,AVpi74XfLJeJML43qZAc,MUK LUKS,"Clothing,Shoes,Women's Shoes,Women's Casual Sh...",35.25,35.25,6,https://www.walmart.com/ip/MUK-LUKS-Womens-Jan...
3,AVpjXyCc1cnluZ0-V-Gj,MUK LUKS,"Clothing,Shoes,Women's Shoes,All Women's Shoes...",24.75,24.75,6,https://www.walmart.com/ip/MUK-LUKS-Womens-Daw...
4,AVphGKLPilAPnD_x1Nrm,MUK LUKS,"Clothing,Shoes,Women's Shoes,All Women's Shoes...",33.00,30.39,6,https://www.walmart.com/ip/MUK-LUKS-Womens-Daw...
5,AVpg91ziilAPnD_xziOo,Soft Ones,"Womens Shoes,Clothing,Women's Shoes,Baby & Kid...",14.00,7.90,5,https://www.walmart.com/ip/MUK-LUKS-Sock-Monke...
6,AVpjGKXyLJeJML43r8BH,MUK LUKS,"MUK LUKS,Womens,Women's Casual Shoes,Clothing,...",24.00,12.79,SMALL,https://www.kohls.com/product/prd-1780800/wome...
7,AVpjGKXyLJeJML43r8BH,MUK LUKS,"MUK LUKS,Womens,Women's Casual Shoes,Clothing,...",24.00,12.79,MEDIUM,https://www.kohls.com/product/prd-1780800/wome...
8,AVpfLXyhilAPnD_xWmNc,MUK LUKS,"Clothing,Shoes,Women's Shoes,All Women's Shoes",59.00,39.88,6,https://www.walmart.com/ip/MUK-LUKS-Women-s-Ma...
9,AVpfeWdJ1cnluZ0-lXYU,MUK LUKS,"Clothing,Shoes,Women's Shoes,Women's Winter Bo...",59.00,47.99,6,https://www.walmart.com/ip/MUK-LUKS-Women-s-Ma...


In [549]:
# After removing duplicates
print("There are {} rows and {} columnns in the dataset".format(shoes_one.shape[0],   shoes_one.shape[1]))
print("There are {} rows and {} columnns in the dataset".format(shoes_two.shape[0],   shoes_two.shape[1]))

There are 10000 rows and 34 columnns in the dataset
There are 8696 rows and 32 columnns in the dataset


In [ ]:
# Merge both dataframes

In [570]:
cols = ['id', 'brand', 'categories', 'prices.amountMax','prices.amountMin', 'prices.size', 'prices.sourceURLs']
combined_shoes = combined_shoes[cols]
#shoes.columns = ['id', 'brand', 'categories', 'Max','Min']

NameError: name 'combined_shoes' is not defined

In [567]:
def parse_url(data):
    domain = urlparse(data).netloc
    domain = domain.split(".")[1:]
    host = ".".join(domain)
    return host

We neeed to extract the merchant name from the url, as we will like to the see the shoe sales spread across the merchants

In [ ]:
combined_shoes['Merchant'] = combined_shoes['prices.sourceURLs'].apply(doubleData)

In [467]:
#shoes['prices.sourceURLs'].str.split("//")[-1].split("/")[0].split("www.")[1].split(".com")[0]

In [468]:
myString.split("//")[-1].split("/")[0].split("www.")[1].split(".com")[0]

'walmart'

In [469]:
#shoes['prices.merchant'].value_counts()

In [470]:
#cols_with_missing = [col for col in shoes.columns if shoes[col].isnull().any()] 
#shoes.drop(cols_with_missing, axis=1, inplace=True)

In [471]:
#shoes.columns

In [545]:
# How many unique brands
print("There are {} unique brands in the dataset".format(combined_shoes['brand'].nunique()))
# How many unique categories
print("There are {} unique categories in the dataset".format(combined_shoes['categories'].nunique()))

NameError: name 'combined_shoes' is not defined

In [475]:
# What is the price range in the dataset
shoes.loc[:,'Range'] = shoes['Max'] - shoes['Min']

In [476]:
#What is the average price of each distinct brand listed?
shoes['Average'] = (shoes['Max'] + shoes['Min'])/2

In [477]:
#Which brands have the highest prices?
highest_prices = shoes.groupby('brand').max()["Max"]
highest = highest_prices[highest_prices == highest_prices.max()]
print("The most expensive brand is {} with a price of {}".format(next(highest.iteritems())[0],next(highest.iteritems())[1]))

The most expensive brand is Red Wing with a price of 289.95


In [478]:
#Which brand is the cheapest the highest prices?
lowest_prices = shoes.groupby('brand').min()["Min"]
lowest = lowest_prices[lowest_prices == lowest_prices.min()]
print("The least expensive brand is {} with a price of {}".format(next(lowest.iteritems())[0],next(lowest.iteritems())[1]))

The least expensive brand is Unbranded with a price of 4.88


In [483]:
type(shoes.nlargest(5, 'Max'))

pandas.core.frame.DataFrame

In [422]:
#Which ones have the widest distribution of prices?
distribution = shoes.groupby('brand').max()["Range"]
#widest = distribution[distribution == distribution.nlargest(3)]
#print("The most expensive brand is {} with a price of {}".format(next(highest.iteritems())[0],next(highest.iteritems())[1]))

ValueError: Can only compare identically-labeled Series objects

In [421]:
widest

brand
Lowa    109.97
Name: Range, dtype: float64

In [ ]:
#Is there a typical price distribution (e.g., normal) across brands or within specific brands?